# Define the ledger and portfolio

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time, timedelta
from zoneinfo import ZoneInfo
from typing import Callable

def compute_payoff(spot, strike, opt_type):
    if opt_type == "call":
        return np.maximum(spot - strike, 0)
    else:

        return np.maximum(strike - spot, 0)

def compute_payoff_vec(spots: np.ndarray,
                       strikes: np.ndarray,
                       opt_types: np.ndarray) -> np.ndarray:
    """
    spots, strikes: float arrays of the same shape
    opt_types: array of 'call' or 'put' strings, same shape
    returns: array of intrinsic payoffs
    """
    # mask of where it's a call
    is_call = (opt_types == "call")
    # intrinsic payoff for calls and for puts
    call_payoff = np.maximum(spots - strikes, 0)
    put_payoff  = np.maximum(strikes - spots, 0)
    # pick the right one for each element
    return np.where(is_call, call_payoff, put_payoff)

class TradeLedger:
    def __init__(self):
        cols = [
            "timestamp","action","instrument_type","underlying",
            "option_type","strike","expiry","quantity","price",
            "signed_quantity","total_cost"
        ]
        self.trades_df = pd.DataFrame(columns=cols)

    def record_trades(self, new_trades: pd.DataFrame):
        """Batch append a DataFrame of trades."""
        print(f"new_trades: {new_trades}")
        self.trades_df = pd.concat([self.trades_df, new_trades], ignore_index=True)
        print(f"self.trades_df: {self.trades_df}")

    def record_trade(self,
                     timestamp: datetime,
                     action: str,
                     instrument_type: str,
                     underlying: str,
                     option_type: str,
                     strike: float,
                     expiry: date,
                     quantity: float,
                     price: float):
        """
        Add a single trade.
        Automatically computes signed_quantity and total_cost.
        """
        signed_qty = quantity if action == "buy" else -quantity
        row = {
            "timestamp": timestamp,
            "action": action,
            "instrument_type": instrument_type,
            "underlying": underlying,
            "option_type": option_type,
            "strike": strike,
            "expiry": expiry,
            "quantity": quantity,
            "price": price,
            "signed_quantity": signed_qty,
            "total_cost": signed_qty * price
        }
        # create 1-row DataFrame and concat
        self.trades_df = pd.concat(
            [self.trades_df, pd.DataFrame([row])],
            ignore_index=True
        )

    @property
    def trades(self):
        """All trades, sorted by timestamp."""
        return self.trades_df.sort_values("timestamp").reset_index(drop=True)

class PortfolioView:
    def __init__(self, ledger: TradeLedger):
        self.ledger = ledger

    def positions_at(self, as_of: datetime) -> pd.DataFrame:
        df = self.ledger.trades_df
        print(f"trades_df: {df}")
        df = df[df["timestamp"] <= as_of]
        print("as_of: ", as_of)
        print(f"df after filtering: {df}")
        grouped = (
            df.groupby(
                ["instrument_type","underlying","option_type","strike","expiry"]
            )["signed_quantity"]
            .sum()
            .reset_index()
        )
        return grouped[grouped["signed_quantity"] != 0]

def expire_trades_vectorized(
    ledger: TradeLedger,
    as_of_date: date,
    price_lookup_fn: Callable[[np.ndarray, date], np.ndarray]
):
    close_dt = datetime.combine(
        as_of_date, time(16,30), tzinfo=ZoneInfo("America/New_York")
    )
    opens = PortfolioView(ledger).positions_at(close_dt)
    opens["expiry_date"] = pd.to_datetime(opens["expiry"]).dt.date
    to_close = opens[opens["expiry_date"] <= as_of_date]
    if to_close.empty:
        return

    spots = price_lookup_fn(to_close["underlying"].values, as_of_date)
    print(f"spots: {spots}")
    payoffs = compute_payoff_vec(
        spots,
        to_close["strike"].values,
        to_close["option_type"].values
    )
    actions = np.where(to_close["signed_quantity"] > 0, "sell", "buy")
    qtys    = to_close["signed_quantity"].abs().values

    batch = pd.DataFrame({
        "timestamp":      close_dt,
        "action":         actions,
        "instrument_type":"option",
        "underlying":     to_close["underlying"].values,
        "option_type":    to_close["option_type"].values,
        "strike":         to_close["strike"].values,
        "expiry":         to_close["expiry"].values,
        "quantity":       qtys,
        "price":          payoffs,
    })
    batch["signed_quantity"] = np.where(batch["action"]=="buy",
                                        batch["quantity"],
                                       -batch["quantity"])
    batch["total_cost"] = batch["signed_quantity"] * batch["price"]

    ledger.record_trades(batch)

# Test the ledger creation and portfolio

In [9]:
td=TradeLedger()
td.record_trade(datetime.now(tz=ZoneInfo("America/New_York")), 'buy', 'option', 'SPY', 'call', 400, datetime.now().date() + timedelta(days=30), 5, 100)
td.record_trade(datetime.now(tz=ZoneInfo("America/New_York")), 'sell', 'option', 'SPY', 'call', 400, datetime.now().date() + timedelta(days=30), 1, 100)
td.record_trade(datetime.now(tz=ZoneInfo("America/New_York")), 'buy', 'option', 'SPY', 'call', 400, datetime.now().date() + timedelta(days=30), 1, 100)

td.record_trade(datetime.now(tz=ZoneInfo("America/New_York")), 'buy', 'option', 'SPY', 'put', 500, datetime.now().date() + timedelta(days=30), 1, 100)
td.trades

pv=PortfolioView(td)
positions=pv.positions_at(datetime.now(tz=ZoneInfo("America/New_York")))
for position in positions.to_dict(orient='records'):
    print(position)


dt=datetime.now()
dt=datetime(dt.year, dt.month, dt.day).date()
#expire_trades(td,dt+timedelta(days=30), random_price_fn)
expire_trades_vectorized(td, dt+timedelta(days=30),random_price_fn)
#td.trades
type(dt+timedelta(days=30))
td.trades

trades_df:                          timestamp action instrument_type underlying  \
0 2025-07-21 09:25:22.470295-04:00    buy          option        SPY   
1 2025-07-21 09:25:22.471898-04:00   sell          option        SPY   
2 2025-07-21 09:25:22.472614-04:00    buy          option        SPY   
3 2025-07-21 09:25:22.473411-04:00    buy          option        SPY   

  option_type strike      expiry quantity price signed_quantity total_cost  
0        call    400  2025-08-20        5   100               5        500  
1        call    400  2025-08-20        1   100              -1       -100  
2        call    400  2025-08-20        1   100               1        100  
3         put    500  2025-08-20        1   100               1        100  
as_of:  2025-07-21 09:25:22.474506-04:00
df after filtering:                          timestamp action instrument_type underlying  \
0 2025-07-21 09:25:22.470295-04:00    buy          option        SPY   
1 2025-07-21 09:25:22.471898-04:00   s

/var/folders/tw/8dy4vcgj7h3fvzr8_jtkzs600000gn/T/ipykernel_1311/1252793266.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trades_df = pd.concat(


,timestamp,action,instrument_type,underlying,option_type,strike,expiry,quantity,price,signed_quantity,total_cost
0,2025-07-21 09:25:22.470295-04:00,buy,option,SPY,call,400,2025-08-20,5,100,5,500
1,2025-07-21 09:25:22.471898-04:00,sell,option,SPY,call,400,2025-08-20,1,100,-1,-100
2,2025-07-21 09:25:22.472614-04:00,buy,option,SPY,call,400,2025-08-20,1,100,1,100
3,2025-07-21 09:25:22.473411-04:00,buy,option,SPY,put,500,2025-08-20,1,100,1,100
4,2025-08-20 16:30:00-04:00,sell,option,SPY,call,400,2025-08-20,5,0.0,-5,-0.0
5,2025-08-20 16:30:00-04:00,sell,option,SPY,put,500,2025-08-20,1,398.232911,-1,-398.232911


In [10]:
dt=datetime.now()
dt=datetime(dt.year, dt.month, dt.day).date()
#expire_trades(td,dt+timedelta(days=30), random_price_fn)
expire_trades_vectorized(td, dt+timedelta(days=30),random_price_fn)

trades_df:                          timestamp action instrument_type underlying  \
0 2025-07-21 09:25:22.470295-04:00    buy          option        SPY   
1 2025-07-21 09:25:22.471898-04:00   sell          option        SPY   
2 2025-07-21 09:25:22.472614-04:00    buy          option        SPY   
3 2025-07-21 09:25:22.473411-04:00    buy          option        SPY   
4        2025-08-20 16:30:00-04:00   sell          option        SPY   
5        2025-08-20 16:30:00-04:00   sell          option        SPY   

  option_type strike      expiry quantity       price signed_quantity  \
0        call    400  2025-08-20        5         100               5   
1        call    400  2025-08-20        1         100              -1   
2        call    400  2025-08-20        1         100               1   
3         put    500  2025-08-20        1         100               1   
4        call    400  2025-08-20        5         0.0              -5   
5         put    500  2025-08-20        1  398

In [8]:
def random_signal_fn(date):
    return {"SPY": 2, "QQQ": -1, "IWM": 1}

def random_price_fn(tickers, date):
    return np.random.random(size=len(tickers)) * 10 + 100

def random_ivol_fn(tickers, date):
    return np.random.random(size=len(tickers)) * 0.2 + 0.1

def random_bs_pricer_fn(spots, ivols, strikes, Ts):
    return np.random.random(size=len(spots)) * 10 + 100

# generate new trades

In [1]:
from datetime import datetime, date, time, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
import numpy as np
from typing import Callable

def generate_new_trades_vectorized(
    ledger: TradeLedger,
    as_of_date: date,
    signal_fn: Callable[[date], dict],
    price_lookup_fn: Callable[[np.ndarray, date], np.ndarray],
    ivol_lookup_fn: Callable[[np.ndarray, date], np.ndarray],
    bs_pricer_fn: Callable[[np.ndarray, np.ndarray, np.ndarray, np.ndarray], np.ndarray],
    holding_period_days: int
):
    """
    1) Get raw signals for today
    2) Filter non‐zero
    3) Vector‐lookup spot, ivol, T
    4) BS‐price entry
    5) Build batch of entry trades & append
    """
    # 1) Timestamp for entries
    entry_ts = datetime.combine(
        as_of_date,
        time(15,59),
        tzinfo=ZoneInfo("America/New_York")
    )

    # 2) Raw signals dict → DataFrame
    raw = signal_fn(as_of_date)       # e.g. {"SPY": 2, "QQQ": -1, ...}
    df = pd.DataFrame(raw.items(), columns=["underlying", "quantity"])
    df = df[df["quantity"] != 0]      # drop zeros
    if df.empty:
        return

    # 3) Vectorized lookups
    tickers = df["underlying"].values
    spots   = price_lookup_fn(tickers, as_of_date)
    ivols   = ivol_lookup_fn(tickers, as_of_date)
    strikes = spots                  # assume ATM
    Ts      = np.full_like(spots, holding_period_days/365, dtype=float)

    print(f"Tickers: {tickers}")
    print(f"df: {df}")
    # 4) BS entry prices
    entry_prices = bs_pricer_fn(spots, strikes, ivols, Ts)
    
    # 5) Compute exit dates
    print(f"as_of_date: {as_of_date}")
    expiries = np.array([(as_of_date + timedelta(days=holding_period_days))] * len(df))

    # 6) Build batch DataFrame
    batch_calls = pd.DataFrame({
        "timestamp":      entry_ts,
        "action":         np.where(df["quantity"]>0, "buy", "sell"),
        "instrument_type":"option",
        "underlying":     tickers,
        "option_type":    "call",       # or "call"/"put" if single‐leg
        "strike":         strikes,
        "expiry":         expiries,
        "quantity":       np.abs(df["quantity"].values),
        "price":          entry_prices/2,
    })
    batch_puts = pd.DataFrame({
        "timestamp":      entry_ts,
        "action":         np.where(df["quantity"]>0, "buy", "sell"),
        "instrument_type":"option",
        "underlying":     tickers,
        "option_type":    "put",       # or "call"/"put" if single‐leg
        "strike":         strikes,
        "expiry":         expiries,
        "quantity":       np.abs(df["quantity"].values),
        "price":          entry_prices/2,
    })
    batch=pd.concat([batch_calls, batch_puts])
    batch["signed_quantity"] = np.where(
        batch["action"]=="buy",
        batch["quantity"],
        -batch["quantity"]
    )
    batch["total_cost"] = batch["signed_quantity"] * batch["price"]
    print(f"batch: {batch}")
    # 7) Append in one go
    ledger.record_trades(batch)
    print(f"ledger.trades: {ledger.trades}")

NameError: name 'TradeLedger' is not defined

In [53]:
portfolio_view=PortfolioView(td)
portfolio_view.positions_at(datetime.now(tz=ZoneInfo("America/New_York")))



#td.trades_df








trades_df:                          timestamp action instrument_type underlying  \
0 2025-07-19 14:44:18.048850-04:00    buy          option        SPY   
1 2025-07-19 14:44:18.050171-04:00   sell          option        SPY   
2 2025-07-19 14:44:18.051030-04:00    buy          option        SPY   
3 2025-07-19 14:44:18.051661-04:00    buy          option        SPY   
4        2025-08-18 16:30:00-04:00   sell          option        SPY   
5        2025-08-18 16:30:00-04:00   sell          option        SPY   

  option_type strike      expiry quantity      price signed_quantity  \
0        call    400  2025-08-18        5        100               5   
1        call    400  2025-08-18        1        100              -1   
2        call    400  2025-08-18        1        100               1   
3         put    500  2025-08-18        1        100               1   
4        call    400  2025-08-18        5        0.0              -5   
5         put    500  2025-08-18        1  398.59442

,instrument_type,underlying,option_type,strike,expiry,signed_quantity
0,option,SPY,call,400,2025-08-18,5
1,option,SPY,put,500,2025-08-18,1


In [56]:
generate_new_trades_vectorized(td, datetime.now(tz=ZoneInfo("America/New_York")).date(), random_signal_fn, random_price_fn, random_ivol_fn, random_bs_pricer_fn, 30)

Tickers: ['SPY' 'QQQ' 'IWM']
df:   underlying  quantity
0        SPY         2
1        QQQ        -1
2        IWM         1
as_of_date: 2025-07-19
batch:                   timestamp action instrument_type underlying option_type  \
0 2025-07-19 09:30:00-04:00    buy          option        SPY        call   
1 2025-07-19 09:30:00-04:00   sell          option        QQQ        call   
2 2025-07-19 09:30:00-04:00    buy          option        IWM        call   
0 2025-07-19 09:30:00-04:00    buy          option        SPY         put   
1 2025-07-19 09:30:00-04:00   sell          option        QQQ         put   
2 2025-07-19 09:30:00-04:00    buy          option        IWM         put   

       strike      expiry  quantity      price  signed_quantity  total_cost  
0  108.206347  2025-08-18         2  51.889633                2  103.779265  
1  105.612637  2025-08-18         1  54.360830               -1  -54.360830  
2  101.811504  2025-08-18         1  50.772846                1   50.77

In [67]:
PortfolioView(td).positions_at(datetime.now(tz=ZoneInfo("America/New_York"))+timedelta(days=31))

trades_df:                           timestamp action instrument_type underlying  \
0  2025-07-19 14:44:18.048850-04:00    buy          option        SPY   
1  2025-07-19 14:44:18.050171-04:00   sell          option        SPY   
2  2025-07-19 14:44:18.051030-04:00    buy          option        SPY   
3  2025-07-19 14:44:18.051661-04:00    buy          option        SPY   
4         2025-08-18 16:30:00-04:00   sell          option        SPY   
5         2025-08-18 16:30:00-04:00   sell          option        SPY   
6         2025-08-18 09:30:00-04:00    buy          option        SPY   
7         2025-08-18 09:30:00-04:00   sell          option        QQQ   
8         2025-08-18 09:30:00-04:00    buy          option        IWM   
9         2025-08-18 09:30:00-04:00    buy          option        SPY   
10        2025-08-18 09:30:00-04:00   sell          option        QQQ   
11        2025-08-18 09:30:00-04:00    buy          option        IWM   
12        2025-07-19 09:30:00-04:00    b

,instrument_type,underlying,option_type,strike,expiry,signed_quantity
1,option,IWM,call,103.856688,2025-09-17,1
3,option,IWM,put,103.856688,2025-09-17,1
4,option,QQQ,call,102.232234,2025-09-17,-1
6,option,QQQ,put,102.232234,2025-09-17,-1
8,option,SPY,call,107.133630,2025-09-17,2
11,option,SPY,put,107.133630,2025-09-17,2


# Create Temporary pricing inputs to test the pnl   

In [43]:
random_td=TradeLedger()
#create a batch of random call, put trades
ntrades=3
tz=ZoneInfo("America/New_York")

entry_ts = datetime.combine(
    date(2025,7,20),
    time(15,59),
)
entry_ts=pd.Timestamp(entry_ts,tz=tz)
entry_ts=np.array([entry_ts]*ntrades)
entry_ts=pd.to_datetime(entry_ts)

action=np.random.choice(["buy","sell"],size=ntrades)
instrument_type="option"
underlying=np.array(["AAPL"]*ntrades)
option_type=np.random.choice(["call","put"],size=ntrades)
strike=np.random.uniform(100,150,size=ntrades)
#expiry=pd.to_datetime(np.random.choice(pd.date_range("2025-07-20",periods=10,freq="B"),size=ntrades))
#expiry=pd.to_datetime([datetime.combine(dt.date(),time(16,15),tzinfo=tz)for dt in expiry])


expiry=np.random.choice(pd.date_range("2025-07-20",periods=10,freq="D"),size=ntrades)
expiry=pd.to_datetime(expiry).normalize()+timedelta(hours=16,minutes=15)
expiry=expiry.tz_localize(tz)

#expiry=np.array(expiry)
#time_aware_expir=[datetime.combine(expiry[i],time(16,15),tzinfo=ZoneInfo("America/New_York"))for i in range(ntrades)]
#quantity=np.random.randint(1,10,size=ntrades)



val_date=datetime.combine(
    date(2025,7,20),
    time(15,59),
    tzinfo=ZoneInfo("America/New_York")
)
val_date=pd.to_datetime(val_date)

texp_days=(expiry-val_date).days/365

texp_seconds=(expiry-val_date).total_seconds()/(365*24*60*60)



"""
# 6) Build batch DataFrame
batch_calls = pd.DataFrame({
    "timestamp":      entry_ts,
    "action":         np.where(df["quantity"]>0, "buy", "sell"),
    "instrument_type":"option",
    "underlying":     tickers,
    "option_type":    "call",       # or "call"/"put" if single‐leg
    "strike":         strikes,
    "expiry":         expiries,
    "quantity":       np.abs(df["quantity"].values),
    "price":          entry_prices/2,
})
random_td.trades

"""








'\n# 6) Build batch DataFrame\nbatch_calls = pd.DataFrame({\n    "timestamp":      entry_ts,\n    "action":         np.where(df["quantity"]>0, "buy", "sell"),\n    "instrument_type":"option",\n    "underlying":     tickers,\n    "option_type":    "call",       # or "call"/"put" if single‐leg\n    "strike":         strikes,\n    "expiry":         expiries,\n    "quantity":       np.abs(df["quantity"].values),\n    "price":          entry_prices/2,\n})\nrandom_td.trades\n\n'

In [44]:
texp_seconds-texp_days
expiry=np.random.choice(pd.date_range("2025-07-20",periods=10,freq="D"),size=ntrades)
expiry=pd.to_datetime(expiry).normalize()+timedelta(hours=16,minutes=15)
expiry=expiry.tz_localize(tz)
expiry

DatetimeIndex(['2025-07-26 16:15:00-04:00', '2025-07-22 16:15:00-04:00',
               '2025-07-22 16:15:00-04:00'],
              dtype='datetime64[ns, America/New_York]', freq=None)

In [14]:
from datetime import datetime, date, time, timedelta
import pandas as pd
import numpy as np
from zoneinfo import ZoneInfo

# Setup
random_td = TradeLedger()
ntrades = 3
tz = ZoneInfo("America/New_York")

# Entry timestamp
entry_ts = pd.Timestamp(datetime(2025, 7, 20, 15, 59), tz=tz)
entry_ts = pd.to_datetime([entry_ts] * ntrades)

# Trade fields
action = np.random.choice(["buy", "sell"], size=ntrades)
instrument_type = "option"
underlying = np.array(["AAPL"] * ntrades)
option_type = np.random.choice(["call", "put"], size=ntrades)
strike = np.random.uniform(100, 150, size=ntrades)
spot=strike

# Expiry timestamps: 10 random days from July 20, 2025 at 4:15 PM
expiry = np.random.choice(pd.date_range("2025-07-20", periods=10, freq="D"), size=ntrades)
expiry = pd.to_datetime(expiry).normalize() + timedelta(hours=16, minutes=15)
expiry = expiry.tz_localize(tz)

# Valuation timestamp
val_date = pd.Timestamp(datetime(2025, 7, 20, 15, 59), tz=tz)

# Time to expiry
texp_days = (expiry - val_date).days / 365
texp_seconds = (expiry - val_date).total_seconds() / (365 * 24 * 60 * 60)

In [5]:
print("Expiry Times:", expiry)
print("Valuation Time:", val_date)
print("Time to Expiry (days):", texp_days)
print("Time to Expiry (fractional):", texp_seconds)

Expiry Times: DatetimeIndex(['2025-07-28 16:15:00-04:00', '2025-07-23 16:15:00-04:00',
               '2025-07-25 16:15:00-04:00'],
              dtype='datetime64[ns, America/New_York]', freq=None)
Valuation Time: 2025-07-20 15:59:00-04:00
Time to Expiry (days): Index([0.021917808219178082, 0.00821917808219178, 0.0136986301369863], dtype='float64')
Time to Expiry (fractional): Index([0.021948249619482496, 0.008249619482496195, 0.013729071537290716], dtype='float64')


In [36]:
import numpy as np
import pandas as pd
import py_vollib.black_scholes
import py_vollib_vectorized



def compute_option_price(spot, strike, cp, ttm, r, q, sigma):
    """
    Vectorized option pricing using py_vollib_vectorized.black_scholes.
    
    Parameters:
        spot: float or array
        strike: float or array
        cp: 'call' or 'put' or array-like
        ttm: float or array (in years)
        r: risk-free rate (scalar or array)
        q: dividend yield (scalar or array)
        sigma: implied volatility (float or array)
    
    Returns:
        np.ndarray of option prices
    """
    cp = np.asarray(cp)
    print(f"cp: {cp}")
    cp = np.where(cp == 'call', 'c', 'p')  # convert to py_vollib format
    print(f"cp: {cp}")

    return py_vollib.black_scholes.black_scholes(
        flag=cp,
        S=np.asarray(spot),
        K=np.asarray(strike),
        t=np.asarray(ttm),
        r=np.asarray(r),
        sigma=np.asarray(sigma)
    )

In [43]:
texp_seconds=1
opt_prices=compute_option_price(spot, strike, option_type, texp_seconds, 0, 0, .4)

results=pd.DataFrame(opt_prices, columns=["spot", "strike", "action", "ttm", "r", "q", "sigma"])
results["spot"]=spot
results["cp"]=option_type
results["strike"]=strike
results["action"]=action
results["ttm"]=texp_seconds
results["r"]=0
results["q"]=0
results["sigma"]=.25
results["opt_price"]=opt_prices
results["pct_price"]=results["opt_price"]/results["spot"]
results

cp: ['put' 'put' 'call']
cp: ['p' 'p' 'c']


,spot,strike,action,ttm,r,q,sigma,cp,opt_price,pct_price
0,138.222314,138.222314,buy,1,0,0,0.25,put,21.910921,0.158519
1,108.820698,108.820698,buy,1,0,0,0.25,put,17.250194,0.158519
2,149.133374,149.133374,buy,1,0,0,0.25,call,23.640536,0.158519


In [45]:
compute_option_price(
    spot=[100, 100],
    strike=[105,95],
    cp=['call', 'put'],
    ttm=[0.5, 0.5],
    r=0.01,
    q=0.0,
    sigma=[0.2, 0.2]
)

cp: ['call' 'put']
cp: ['c' 'p']


,Price
0,3.798807
1,3.173880
